In [ ]:
import streamlit as st
import cv2
import numpy as np
from PIL import Image, ImageOps
from keras.models import load_model  # TensorFlow is required for Keras to work

In [ ]:
pwd

In [ ]:
#get the prediction from loaded model
model = load_model("pages/TM/model.savedmodel", compile=False)

In [ ]:
# Load the labels
class_names = open("pages/TM/TM_labels.txt", "r").readlines()

In [ ]:
# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Replace this with the path to your image
image = Image.open("pages/TM/B_test2.jpg").convert("RGB")

# resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Load the image into the array
data[0] = normalized_image_array

# Predicts the model
prediction = model.predict(data)
index = np.argmax(prediction)
class_name = class_names[index]
confidence_score = prediction[0][index]

# Print prediction and confidence score
print("Letter:", class_name[2:], end="")
print("Confidence Score:", confidence_score)

## testing the model creation

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory

In [ ]:
path = "generated_images"

In [ ]:
train_ds = image_dataset_from_directory(
  path,
  labels = "inferred",
  label_mode = "categorical",
  seed=42,
  image_size=(200, 200),
  batch_size=16,
  validation_split=0.3,
  subset = 'training')

# We define a second one for the test data

validation_ds = image_dataset_from_directory(
  path,
  labels = "inferred",
  label_mode = "categorical",
  seed=42,
  image_size=(200, 200),
  batch_size=16,
  validation_split=0.3,
  subset = 'validation')

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers

In [ ]:
MODEL = "model_1"

modelCheckpooint = callbacks.ModelCheckpoint("{}.h5".format(MODEL), monitor="val_loss", verbose=0, save_best_only=True)

LRreducer = callbacks.ReduceLROnPlateau(monitor="val_loss", factor = 0.1, patience=3, verbose=1, min_lr=0)

EarlyStopper = callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=0, restore_best_weights=True)

In [ ]:
model = Sequential()

model.add(layers.Rescaling(1./255, input_shape = (200, 200, 3)))
model.add(layers.Conv2D(filters = 32, kernel_size = (3,3), activation="relu", padding = "same"))
model.add(layers.MaxPooling2D(pool_size=(2, 2), padding = "same") )


model.add(layers.Conv2D(filters = 32, kernel_size = (3,3), activation="relu", padding = "same"))
model.add(layers.MaxPooling2D(pool_size=(2, 2), padding = "same") )


model.add(layers.Conv2D(filters = 64, kernel_size = (3,3), activation="relu", padding = "same"))
model.add(layers.MaxPooling2D(pool_size=(2, 2), padding = "same") )

model.add(layers.Conv2D(filters = 128, kernel_size = (3,3), activation="relu", padding = "same"))
model.add(layers.MaxPooling2D(pool_size=(2, 2), padding = "same") )

model.add(layers.Flatten())

# Here we flatten our data to end up with just one dimension

model.add(layers.Dense(64, activation="relu"))

model.add(layers.Dropout(0.5))

model.add(layers.Dense(26, activation="softmax"))

model.summary()

In [ ]:
adam = optimizers.Adam(learning_rate = 0.001)
model.compile(loss='categorical_crossentropy',
              optimizer= adam,
              metrics=['accuracy'])

In [ ]:
history = model.fit(
        train_ds,
        epochs=40,
        validation_data=validation_ds,
        batch_size = 64,
        callbacks = [modelCheckpooint, LRreducer, EarlyStopper])

In [ ]:
from tensorflow.keras.saving import load_model
import matplotlib.pyplot as plt
import cv2
import numpy as np

In [ ]:
model_imported = load_model('model_1.h5')

In [ ]:
test_image = cv2.imread('generated_images/A_annotated_image.jpg')
plt.imshow(test_image)

In [ ]:
test_image.shape

In [ ]:
test_image_exp = np.expand_dims(test_image, axis=0)
test_image_exp.shape

In [ ]:
y_test = np.zeros(26)
y_test[0] = 1 #as the test image is A
y_image_exp = np.expand_dims(y_test, axis=0)
y_image_exp.shape

In [ ]:
model_imported.evaluate(test_image_exp, y_image_exp)

# Update jpg image with landmarks and crop it

### first step we display the input image

In [ ]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt

In [ ]:
image = cv2.imread('raw_data/Kaggle2/data/C/C192.jpg')
plt.imshow(image)

### add landmarks

In [ ]:
# Initialize MediaPipe Hand module
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands.Hands()

# Convert image to RGB and process with Hand module
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
results = mp_hands.process(image_rgb)

# Draw landmarks on the image
if results.multi_hand_landmarks:
    for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp.solutions.hands.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2)
        )

# Save the image with landmarks
cv2.imwrite('raw_data/landmarked.jpg', image)

In [ ]:
image_landmark = cv2.imread('raw_data/landmarked.jpg')
plt.imshow(image_landmark)

### crop the image based on landmarks

In [13]:
landmark_list = []
if results.multi_hand_landmarks:
    for hand_landmarks in results.multi_hand_landmarks:
        for landmark in hand_landmarks.landmark:
            landmark_list.append((landmark.x, landmark.y))
            
#calculate the bounding box
x_coordinates = [landmark[0] for landmark in landmark_list]
y_coordinates = [landmark[1] for landmark in landmark_list]
_extend = 0.1
min_x = max(min(x_coordinates)-_extend,0)
max_x = min(max(x_coordinates)+_extend,1)
min_y = max(min(y_coordinates)-_extend,0)
max_y = min(max(y_coordinates)+_extend,1)

# crop the image
cropped_image = image_landmark[int(min_y * image.shape[0]):int(max_y * image.shape[0]), int(min_x * image.shape[1]):int(max_x * image.shape[1])]

cv2.imwrite('raw_data/landmarked_cropped.jpg', cropped_image)

NameError: name 'image_landmark' is not defined

In [ ]:
image_landmark_cropped = cv2.imread('raw_data/landmarked_cropped.jpg')
plt.imshow(image_landmark_cropped)

In [ ]:
image_landmark_cropped.shape

In [ ]:
image_landmark_cropped_resize = cv2.resize(image_landmark_cropped, (300, 300))
plt.imshow(image_landmark_cropped_resize)

In [ ]:
image_landmark_cropped_resize.shape

### making a loop for couple of folders

In [3]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import os
import numpy as np
import gc
import time
count = 0

In [2]:
path = 'raw_data/Kaggle2/data'
output_path = 'raw_data/Kaggle2/landmarked'
for folder in os.listdir(path):
    #create a new folder for future output
    os.mkdir(os.path.join(output_path, folder))
    for file in os.listdir(os.path.join(path, folder)):
        if file.endswith(".jpg"):
            
            image = cv2.imread(os.path.join(path, os.path.join(folder, file)))

            # Initialize MediaPipe Hand module
            mp_drawing = mp.solutions.drawing_utils
            mp_hands = mp.solutions.hands.Hands()

            # Convert image to RGB and process with Hand module
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = mp_hands.process(image_rgb)

            # Draw landmarks on the image
            landmark_list = []
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                     mp_drawing.draw_landmarks(
                        image,
                        hand_landmarks,
                        mp.solutions.hands.HAND_CONNECTIONS,
                        mp_drawing.DrawingSpec(color = (0, 0, 255), thickness=2, circle_radius=2),
                        mp_drawing.DrawingSpec(color = (255, 255, 255), thickness=2))
                     for landmark in hand_landmarks.landmark:
                        landmark_list.append((landmark.x, landmark.y))

            # QC the landmark setting process and skip image if did not work
            if len(landmark_list) < 1: continue
                
            #calculate the bounding box
            x_coordinates = [landmark[0] for landmark in landmark_list]
            y_coordinates = [landmark[1] for landmark in landmark_list]
            
            # extending, yet making sure it's within [0,1] canvas
            _extend = 0.1
            min_x = max(min(x_coordinates)-_extend,0)
            max_x = min(max(x_coordinates)+_extend,1)
            min_y = max(min(y_coordinates)-_extend,0)
            max_y = min(max(y_coordinates)+_extend,1)

            # crop the image
            cropped_image = image[int(min_y * image.shape[0]):int(max_y * image.shape[0]), int(min_x * image.shape[1]):int(max_x * image.shape[1])]

            if np.logical_and.reduce([cropped_image.shape[0]>=50, cropped_image.shape[1]>=50, len(landmark_list)>15]):
                image_export = cv2.resize(cropped_image, (300, 300))
                cv2.imwrite(f'raw_data/Kaggle2/landmarked/{folder}/{file}', image_export)
                gc.collect()
            else:
                count+=1
                print("mising", count)

FileExistsError: [Errno 17] File exists: 'raw_data/Kaggle2/landmarked/J'

### looks like have to go  folder by folder((

In [17]:
%%time
folder_list = ['H', 'I']
path = 'raw_data/Kaggle2/data'
output_path = 'raw_data/Kaggle2/landmarked'

# Initialize MediaPipe Hand module
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands.Hands()

for folder in folder_list:
    os.mkdir(os.path.join(output_path, folder))
    for file in os.listdir(os.path.join(path, folder)):
        if file.endswith(".jpg"):
            
            image = cv2.imread(os.path.join(os.path.join(path, folder), file))

            # Convert image to RGB and process with Hand module
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = mp_hands.process(image_rgb)

            # Draw landmarks on the image
            landmark_list = []
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                     mp_drawing.draw_landmarks(
                        image,
                        hand_landmarks,
                        mp.solutions.hands.HAND_CONNECTIONS,
                        mp_drawing.DrawingSpec(color = (0, 0, 255), thickness=2, circle_radius=2),
                        mp_drawing.DrawingSpec(color = (255, 255, 255), thickness=2))
                     for landmark in hand_landmarks.landmark:
                        landmark_list.append((landmark.x, landmark.y))

            # QC the landmark setting process and skip image if did not work
            if len(landmark_list) < 1: continue
                
            #calculate the bounding box
            x_coordinates = [landmark[0] for landmark in landmark_list]
            y_coordinates = [landmark[1] for landmark in landmark_list]
            
            # extending, yet making sure it's within [0,1] canvas
            _extend = 0.1
            min_x = max(min(x_coordinates)-_extend,0)
            max_x = min(max(x_coordinates)+_extend,1)
            min_y = max(min(y_coordinates)-_extend,0)
            max_y = min(max(y_coordinates)+_extend,1)

            # crop the image
            cropped_image = image[int(min_y * image.shape[0]):int(max_y * image.shape[0]), int(min_x * image.shape[1]):int(max_x * image.shape[1])]

            if np.logical_and.reduce([cropped_image.shape[0]>=50, cropped_image.shape[1]>=50, len(landmark_list)>15]):
                image_export = cv2.resize(cropped_image, (300, 300))
                cv2.imwrite(f'{os.path.join(output_path, folder)}/{file}', image_export)
                gc.collect()
#                 time.sleep(0.1)

CPU times: user 14min 26s, sys: 16.2 s, total: 14min 42s
Wall time: 14min 15s
